In [ ]:
!nvidia-smi

Sun Jul 25 14:06:33 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#config consts
DATASET =     {'train':'train.tar.gz',
               'test':'test.tar.gz',
               'train_mask':'train_mask.tar.gz'}

METADATA=     {'image_level':'train_image_level.csv',
               'study_level':'train_study_level.csv',
               'df_meta':'df_meta.csv',
               'df_fold_rand830':'df_fold_rand830.csv',
               'train_dup':'duplicate.txt',
               'sample_sub':'sample_submission.csv'}

CFGMODEL_DIR_DICT = {'B3_512':'effb3-full-512-mask-v8/',
                     'B4_512':'effb4-full-512-mask/',
                     'B5_600':'effb5-600-mask/',
                     'B5_640':'effb5-640-mask/',
                     'D201_640':'d201-640-mask/',
                     'B7_768':'effb7-768-mask/'}

INPUT_DIR ='/content/drive/My\ Drive/kaggle/covid19-det/input/'

OUTPUT_DIR = {'BSL':'/content/drive/My\ Drive/kaggle/covid19-det/output/',
              'NORM':'/content/drive/My Drive/kaggle/covid19-det/output/'}

IMPORT_DIR = '/content/drive/My Drive/kaggle/covid19-det/nbs/py/'

HENGCK_IM_DIR=IMPORT_DIR+'hengck_code/dummy_01q/'

WORK_DIR='/content/'

DATASET_DIR_DICT = {'256': INPUT_DIR+'256_jpg/',
                    '512': INPUT_DIR+'512_jpg/',
                    '600': INPUT_DIR+'600_jpg/',
                    '640': INPUT_DIR+'640_jpg/',
                    '768': INPUT_DIR+'768_jpg/'}

EXPERIMENT='SZ768_EFNB7_5FOLDS'
EXPERIMENT_DIR = OUTPUT_DIR['BSL'] + EXPERIMENT+'/'
CFGMODEL_DIR = CFGMODEL_DIR_DICT['B7_768']
MASK_SIZE=(48,48)
DATASET_DIR = DATASET_DIR_DICT['768']
METADATA_DIR = INPUT_DIR+'metadata/'
FOLDS_SET=[0,1,2,3,4]
ENSEMBLE_CSV='/768_b7_5f.csv'
INITIAL_CHECKPOINTS=['best_model.pth' for i in range(5)]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
if 0:
  !cp -r {OUTPUT_DIR['BSL']+'mpack'} '/content'
  !tar zcvf mpack.tar.gz '/content/mpack/'
  !cp /content/mpack.tar.gz {OUTPUT_DIR['BSL']}

In [ ]:
def copy_dataset(ds_dict, ds_dir, work_dir):
  for record in ds_dict:
    print('copy', ds_dir+ds_dict[record], ' to', work_dir)
    !cp {ds_dir+ds_dict[record]} {work_dir}
    print('mkdir',work_dir+record)
    !mkdir {work_dir+record}
    print ('tar -xzf',work_dir+ds_dict[record],'-C',work_dir+record)
    !tar -xzf  {work_dir+ds_dict[record]} -C {work_dir+record}
    print ('rm ',work_dir+ds_dict[record])
    !rm {work_dir+ds_dict[record]}
def copy_metadata(md_dict,md_dir,work_dir):
  for record in md_dict:
    print('copy ', md_dir+md_dict[record],' to ',work_dir)
    !cp {md_dir+md_dict[record]} {work_dir}

copy_dataset(DATASET,DATASET_DIR, WORK_DIR)
copy_metadata(METADATA,METADATA_DIR,WORK_DIR)
!ls /content/

copy /content/drive/My\ Drive/kaggle/covid19-det/input/768_jpg/train.tar.gz  to /content/
mkdir /content/train
tar -xzf /content/train.tar.gz -C /content/train
rm  /content/train.tar.gz
copy /content/drive/My\ Drive/kaggle/covid19-det/input/768_jpg/test.tar.gz  to /content/
mkdir /content/test
tar -xzf /content/test.tar.gz -C /content/test
rm  /content/test.tar.gz
copy /content/drive/My\ Drive/kaggle/covid19-det/input/768_jpg/train_mask.tar.gz  to /content/
mkdir /content/train_mask
tar -xzf /content/train_mask.tar.gz -C /content/train_mask
rm  /content/train_mask.tar.gz
copy  /content/drive/My\ Drive/kaggle/covid19-det/input/metadata/train_image_level.csv  to  /content/
copy  /content/drive/My\ Drive/kaggle/covid19-det/input/metadata/train_study_level.csv  to  /content/
copy  /content/drive/My\ Drive/kaggle/covid19-det/input/metadata/df_meta.csv  to  /content/
copy  /content/drive/My\ Drive/kaggle/covid19-det/input/metadata/df_fold_rand830.csv  to  /content/
copy  /content/drive/My\ D

In [ ]:
import sys
sys.path.append(HENGCK_IM_DIR)
sys.path.append(HENGCK_IM_DIR+CFGMODEL_DIR)

!pip install pydicom
!pip install madgrad
!pip install timm

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
from siim import *


from model import *
from dataset import *
from common import *

     |████████████████████████████████| 1.9 MB 13.7 MB/s 
     |████████████████████████████████| 376 kB 13.2 MB/s 


In [ ]:
# start here ! ###################################################################################
def probability_to_df_study(df_valid, probability):
    df_study = pd.DataFrame()
    df_study.loc[:,'id'] = df_valid.study + '_study'
    for i in range(num_study_label):
        df_study.loc[:,study_name_to_predict_string[study_label_to_name[i]]]=probability[:,i]

    df_study = df_study.groupby('id', as_index=False).mean()
    df_study.loc[:, 'PredictionString'] = \
           'negative '      + df_study.negative.apply(lambda x: '%0.6f'%x)      + ' 0 0 1 1' \
        + ' typical '       + df_study.typical.apply(lambda x: '%0.6f'%x)       + ' 0 0 1 1' \
        + ' indeterminate ' + df_study.indeterminate.apply(lambda x: '%0.6f'%x) + ' 0 0 1 1' \
        + ' atypical '      + df_study.atypical.apply(lambda x: '%0.6f'%x)      + ' 0 0 1 1'

    df_study = df_study[['id','PredictionString']]
    return df_study

def probability_to_df_image(df_valid, probability, box):
    df_image = pd.DataFrame({'id':[],'PredictionString':[]})
    return df_image

def do_predict(net, valid_loader, tta=['flip','scale']): #flip

    valid_probability = []
    valid_num = 0

    start_timer = timer()
    for t, batch in enumerate(valid_loader):
        batch_size = len(batch['index'])
        image  = batch['image'].cuda()
        onehot = batch['onehot']
        label  = onehot.argmax(-1)

        #<todo> TTA
        net.eval()
        with torch.no_grad():
            probability = []
            logit, mask = net(image)
            probability.append(F.softmax(logit,-1))

            if 'flip' in tta:
                logit, mask = net(torch.flip(image,dims=(3,)))
                probability.append(F.softmax(logit,-1))

            if 'scale' in tta:
                # size=None, scale_factor=None, mode='nearest', align_corners=None, recompute_scale_factor=None):
                logit, mask = net(F.interpolate(image, scale_factor=1.33, mode='bilinear', align_corners=False))
                probability.append(F.softmax(logit,-1))

            #--------------
            probability = torch.stack(probability,0).mean(0)

        valid_num += batch_size
        valid_probability.append(probability.data.cpu().numpy())
        print('\r %8d / %d  %s' % (valid_num, len(valid_loader.dataset), time_to_str(timer() - start_timer, 'sec')),
              end='', flush=True)

    assert(valid_num == len(valid_loader.dataset))
    print('')

    probability = np.concatenate(valid_probability)
    return probability

In [ ]:
def run_submit():
    for fold in FOLDS_SET:
        out_dir = OUTPUT_DIR['NORM']+EXPERIMENT+'/fold%d'%fold
        initial_checkpoint = out_dir + '/checkpoint/'+INITIAL_CHECKPOINTS[fold] # None #

        if 1:

            ## setup  ----------------------------------------
            #mode = 'local'
            mode = 'remote'

            submit_dir = out_dir + '/valid/%s'%(mode)
            os.makedirs(submit_dir, exist_ok=True)

            log = Logger()
            log.open(out_dir + '/log.submit.txt', mode='a')
            log.write('\n--- [START %s] %s\n\n' % (IDENTIFIER, '-' * 64))
            log.write('\t%s\n' % COMMON_STRING)
            log.write('\n')

            #
            ## dataset ------------------------------------

            if 'remote' in mode: #1263
                df_valid = make_fold('test')

            if 'local' in mode: #1276 #1256
                df_train, df_valid = make_fold('train-%d' % fold)


            valid_dataset = SiimDataset(df_valid)
            valid_loader  = DataLoader(
                valid_dataset,
                sampler = SequentialSampler(valid_dataset),
                batch_size  = 10,#128, #
                drop_last   = False,
                num_workers = 2,
                pin_memory  = True,
                collate_fn  = null_collate,
            )
            log.write('mode : %s\n'%(mode))
            log.write('valid_dataset : \n%s\n'%(valid_dataset))

            ## net ----------------------------------------
            if 1:
                net = Net().cuda()
                net.load_state_dict(torch.load(initial_checkpoint)['state_dict'], strict=True)

                #---
                start_timer = timer()
                probability = do_predict(net, valid_loader)
                log.write('time %s \n' % time_to_str(timer() - start_timer, 'min'))
                log.write('probability %s \n' % str(probability.shape))

                np.save(submit_dir + '/probability.npy',probability)
                df_valid.to_csv(submit_dir + '/df_valid.csv', index=False)

            else:
                probability = np.load(submit_dir + '/probability.npy')

            #----
            df_study = probability_to_df_study(df_valid, probability)
            df_image = probability_to_df_image(df_valid, None, None)
            df_submit = pd.concat([df_study,df_image])
            df_submit.to_csv(submit_dir + '/submit.csv', index=False)

            log.write('submit_dir : %s\n' % (submit_dir))
            log.write('initial_checkpoint : %s\n' % (initial_checkpoint))
            log.write('df_submit : %s\n' % str(df_submit.shape))
            log.write('%s\n' % str(df_submit))
            log.write('\n')

            if 'local' in mode:
                onehot = df_valid[study_name_to_label.keys()].values
                truth = onehot.argmax(-1)
                predict = probability.argsort(-1)[:,::-1]

                loss = np_loss_cross_entropy(probability, truth)
                topk = (predict == truth.reshape(-1, 1))
                acc  = topk[:, 0]
                topk = topk.mean(0).cumsum()
                acc  = [acc[truth == i].mean() for i in range(num_study_label)]

                # ---
                map = np_metric_map_curve_by_class(probability, truth)

                # ---
                log.write('loss : %f\n' % (loss))
                log.write('topk : %s\n' % str(topk))
                log.write('map(mean) : %f\n' % map.mean())
                log.write('          : %f\n' % ((4 / 6)* map.mean()))
                for i in range(num_study_label):
                    l = study_label_to_name[i]
                    log.write('%d %30s : %0.5f\n' % (i,l,map[i]))
                log.write('\n\n')
        #exit(0)

In [ ]:
def run_remote_ensemble():
    out_dir = OUTPUT_DIR['NORM']+EXPERIMENT
    log = Logger()
    log.open(out_dir + '/log.submit.txt', mode='a')
    log.write('\n--- [START %s] %s\n\n' % (IDENTIFIER, '-' * 64))
    log.write('\t%s\n' % COMMON_STRING)
    log.write('\n')

    submit_dir=[]
    for fold in FOLDS_SET:
      submit_dir.append(out_dir+'/fold%d/valid/remote/'%fold)
    
    probability=0
    for d in submit_dir:
        p = np.load(d + '/probability.npy')
        probability += p**0.5
    probability = probability/len(submit_dir)


    #----
    df_valid = pd.read_csv(submit_dir[1] + '/df_valid.csv')

    df_study  = probability_to_df_study(df_valid, probability)
    df_image  = probability_to_df_image(df_valid, None, None)
    df_submit = pd.concat([df_study, df_image])
    df_submit.to_csv(out_dir + ENSEMBLE_CSV, index=False)

    log.write('submit_dir : %s\n' % (submit_dir))
    log.write('df_submit : %s\n' % str(df_submit.shape))
    log.write('%s\n' % str(df_submit))
    log.write('\n')

In [ ]:
run_submit()

In [ ]:
run_remote_ensemble()